# [LRU Cache](https://leetcode.com/problems/lru-cache/description/)

Design a data structure to store key-value pairs with fast access and update. When capacity is exceeded, evict the least recently used item.
## Strategy
> Without capacity, this would be a simple hash, so you'll need another data structure. The O(1) requirement hints that additional structure must also have O(1) ability, like a linked list.
- Use a [doubly linked list](../resources/linked-list.ipynb) to keep track of usage order (most recent at head).
- Use a hash map for O(1) access to nodes.
- On get, move accessed node to head.
- On put, insert new node at head, remove tail if over capacity.

## Remember
- When removing a doubling linked list element with dummy nodes, you can use a generic code to remove any element

# LRU Cache Step-by-Step Walkthrough

Let's trace through the STL-based LRU Cache with a concrete example: `LRUCache cache(2)` (capacity = 2).

## Initial State
```cpp
LRUCache cache(2);
```

**State:**
- `capacity = 2`
- `lru = []` (empty list)
- `dic = {}` (empty map)

---

## Step 1: `cache.put(1, 100)`

**Operation:** Insert key=1, value=100

### Execution Flow:
1. **Check if key exists:** `dic.find(1) == dic.end()` → No, key doesn't exist
2. **Skip removal step** (key not found)
3. **Add to front:** `lru.push_front({1, 100})`
4. **Update map:** `dic[1] = lru.begin()`
5. **Check capacity:** `dic.size() = 1 <= 2` → OK

**Resulting State:**
```
lru: [(1,100)]
     ↑ front (most recent)

dic: {1 → iterator_pointing_to_(1,100)}
```

---

## Step 2: `cache.put(2, 200)`

**Operation:** Insert key=2, value=200

### Execution Flow:
1. **Check if key exists:** `dic.find(2) == dic.end()` → No
2. **Skip removal step**
3. **Add to front:** `lru.push_front({2, 200})`
4. **Update map:** `dic[2] = lru.begin()`
5. **Check capacity:** `dic.size() = 2 <= 2` → OK

**Resulting State:**
```
lru: [(2,200), (1,100)]
     ↑ front   ↑ back
     (most recent) (least recent)

dic: {
    1 → iterator_pointing_to_(1,100),
    2 → iterator_pointing_to_(2,200)
}
```

---

## Step 3: `cache.get(1)`

**Operation:** Get key=1 (should return 100 and move to front)

### Execution Flow:
1. **Find key:** `auto it = dic.find(1)` → Found!
2. **Extract value:** `int value = it->second->second`
   - `it->second` = iterator pointing to `(1,100)` in list
   - `it->second->second` = `100` (the value part of the pair)
3. **Remove from current position:** `lru.erase(it->second)`
4. **Add to front:** `lru.push_front({1, 100})`
5. **Update map:** 
   - `dic.erase(it)` → Remove old mapping
   - `dic[1] = lru.begin()` → Point to new front position
6. **Return value:** `return 100`

**Resulting State:**
```
lru: [(1,100), (2,200)]
     ↑ front   ↑ back
     (most recent) (least recent)

dic: {
    1 → iterator_pointing_to_(1,100)_at_front,
    2 → iterator_pointing_to_(2,200)_at_back
}
```

**Key Point:** Notice how `(1,100)` moved from back to front!

---

## Step 4: `cache.put(3, 300)`

**Operation:** Insert key=3, value=300 (this will trigger eviction!)

### Execution Flow:
1. **Check if key exists:** `dic.find(3) == dic.end()` → No
2. **Skip removal step**
3. **Add to front:** `lru.push_front({3, 300})`
4. **Update map:** `dic[3] = lru.begin()`
5. **Check capacity:** `dic.size() = 3 > 2` → **EVICTION NEEDED!**

### Eviction Process:
6. **Find LRU item:** `lru.rbegin()->first`
   - `lru.rbegin()` points to last element = `(2,200)`
   - `lru.rbegin()->first` = `2` (the key)
7. **Remove from map:** `dic.erase(dic.find(2))`
8. **Remove from list:** `lru.pop_back()`

**Resulting State:**
```
lru: [(3,300), (1,100)]
     ↑ front   ↑ back
     (most recent) (least recent)

dic: {
    1 → iterator_pointing_to_(1,100),
    3 → iterator_pointing_to_(3,300)
}
```

**Key Point:** `(2,200)` was evicted because it was least recently used!

---

## Step 5: `cache.get(2)`

**Operation:** Try to get key=2 (should return -1)

### Execution Flow:
1. **Find key:** `auto it = dic.find(2)` → `it == dic.end()` → Not found!
2. **Return -1**

**State:** Unchanged (no key=2 exists)

---

## Step 6: `cache.get(1)`

**Operation:** Get key=1 again

### Execution Flow:
1. **Find key:** `dic.find(1)` → Found!
2. **Extract value:** `100`
3. **Remove and re-add:** Move `(1,100)` to front
4. **Return:** `100`

**Resulting State:**
```
lru: [(1,100), (3,300)]
     ↑ front   ↑ back

dic: {
    1 → iterator_pointing_to_(1,100)_at_front,
    3 → iterator_pointing_to_(3,300)_at_back
}
```

---

## Key Insights

### Why This Works:
1. **O(1) List Operations:** `std::list` provides O(1) insert/delete when you have an iterator
2. **Iterator Storage:** Storing iterators in the map eliminates O(n) search through the list
3. **Front = Recent:** New items and accessed items go to front
4. **Back = LRU:** Eviction always happens from the back

### The Magic of Iterators:
```cpp
// Instead of searching through list (O(n)):
for (auto& pair : lru) {
    if (pair.first == key) { /* found it */ }
}

// We directly jump to position (O(1)):
auto listIterator = dic[key];  // Get iterator from map
lru.erase(listIterator);       // Direct removal
```


```c++

```c++
/**
 * LRU Cache Implementation Using STL Containers
 * 
 * Design Strategy:
 * - std::list<pair<int,int>>: Maintains insertion order, O(1) insert/delete
 * - unordered_map<int, iterator>: Maps keys to list positions for O(1) lookup
 * 
 * Data Structure Layout:
 * list: [most recent] ←→ ... ←→ [least recent]
 *       front()                    back()
 * 
 * Key Insight: Store iterators in the map to avoid O(n) list traversal
 */
class LRUCache {
public:
    int capacity;                                                    // Maximum cache size
    unordered_map<int, list<pair<int, int>>::iterator> dic;        // key → iterator mapping
    list<pair<int, int>> lru;                                      // Doubly-linked list of {key, value} pairs
    
    /**
     * Constructor: Initialize cache with given capacity
     * @param capacity: Maximum number of key-value pairs to store
     */
    LRUCache(int capacity) { 
        this->capacity = capacity; 
    }
    
    /**
     * Get value by key and mark as recently used
     * 
     * Algorithm:
     * 1. Check if key exists in map
     * 2. If found: extract value, remove from current position, add to front
     * 3. Update map with new iterator position
     * 
     * Time Complexity: O(1)
     * 
     * @param key: Key to lookup
     * @return: Value if found, -1 if not found
     */
    int get(int key) {
        // Step 1: Check if key exists in hash map
        auto it = dic.find(key);
        if (it == dic.end()) {
            return -1;  // Key not found
        }
        
        // Step 2: Extract value from the list node
        // it->second is the list iterator
        // it->second->second is the value (second element of the pair)
        int value = it->second->second;
        
        // Step 3: Remove from current position in list
        // This is O(1) because we have the exact iterator
        lru.erase(it->second);
        
        // Step 4: Add to front (mark as most recently used)
        lru.push_front({key, value});
        
        // Step 5: Update map with new iterator position
        // Must erase old entry first, then insert new one
        dic.erase(it);                  // Remove old key→iterator mapping
        dic[key] = lru.begin();         // Map key to new front position
        
        return value;
    }
    
    /**
     * Insert or update key-value pair
     * 
     * Algorithm:
     * 1. If key exists: remove old entry
     * 2. Add new entry to front
     * 3. If over capacity: remove least recently used (back)
     * 
     * Time Complexity: O(1)
     * 
     * @param key: Key to insert/update
     * @param value: Value to associate with key
     */
    void put(int key, int value) {
        // Step 1: Check if key already exists
        auto it = dic.find(key);
        if (dic.find(key) != dic.end()) {           // Note: Redundant find() call - could optimize
            // Key exists: remove old entry from both list and map
            lru.erase(it->second);                  // Remove from list using stored iterator
            dic.erase(it);                          // Remove from map
        }
        
        // Step 2: Add new entry to front of list (most recently used position)
        lru.push_front({key, value});
        
        // Step 3: Update map to point to new front element
        dic[key] = lru.begin();                     // lru.begin() points to front
        
        // Step 4: Handle capacity constraint
        if (dic.size() > capacity) {
            // Remove least recently used item (back of list)
            
            // Get key of last element using reverse iterator
            // lru.rbegin() points to last element
            // lru.rbegin()->first is the key of last pair
            auto lastKey = lru.rbegin()->first;
            
            // Remove from map first (need key for lookup)
            auto lastIt = dic.find(lastKey);
            dic.erase(lastIt);
            
            // Remove from back of list
            lru.pop_back();
        }
    }
};
```

## Solution



In [ ]:
class Node {
  constructor(key?: number, val?: number) {
    this.key = key ?? null
    this.val = val ?? null
    this.next = null
    this.prev = null
  }
}

class LRUCache {
  constructor(capacity: number) {
      this.map = new Map<number, Node>()
      this.capacity = capacity
      this.head = new Node()
      this.tail = new Node()
      this.head.next = this.tail
      this.tail.prev = this.head
  }

  moveToFront(key: number) {
      const node: Node = this.map.get(key)

      // head node can remain where it is
      if(this.head.next !== node) {
        // remove node
        node.prev.next = node.next
        node.next.prev = node.prev
        
        node.next = this.head.next
        node.prev = this.head
        this.head.next.prev = node
        this.head.next = node
      }
  }

  get(key: number): number {
    if (this.map.has(key)) {
        this.moveToFront(key)
        return this.map.get(key).val
    }

    return -1
  }

  put(key: number, value: number): void {
    // just update if it already exists
    if (this.map.has(key)) {
      this.map.get(key).val = value
      this.moveToFront(key)
    } else {
      // remove the LRU
      if ((this.map.size + 1) > this.capacity) {
        // cull
        const lru: Node = this.tail.prev
        lru.prev.next = this.tail
        this.tail.prev = lru.prev

        // remove from the map
        this.map.delete(lru.key)
      }

      // store in front
      const node: Node = new Node(key, value)
      node.prev = this.head
      node.next = this.head.next
      this.head.next.prev = node
      this.head.next = node

      // store in map
      this.map.set(key, node)
    }
  }
}

## Test Cases



In [48]:
import { assertEquals } from "https://deno.land/std/testing/asserts.ts";

Deno.test("LRU Cache: put key 1", () => {
  const cache = new LRUCache(2);
  cache.put(1, 1);
  assertEquals(cache.get(1), 1);
});

Deno.test("LRU Cache: put key 2", () => {
  const cache = new LRUCache(2);
  cache.put(1, 1);
  cache.put(2, 2);
  assertEquals(cache.get(2), 2);
});

Deno.test("LRU Cache: evict least recently used key", () => {
  const cache = new LRUCache(2);
  cache.put(1, 1);
  cache.put(2, 2);
  cache.get(1);          // Access key 1
  cache.put(3, 3);       // Evicts key 2
  assertEquals(cache.get(2), -1);
});

Deno.test("LRU Cache: update value of existing key", () => {
  const cache = new LRUCache(2);
  cache.put(1, 1);
  cache.put(1, 10);
  assertEquals(cache.get(1), 10);
});

Deno.test("LRU Cache: evict after capacity exceeded", () => {
  const cache = new LRUCache(2);
  cache.put(1, 1);
  cache.put(2, 2);
  cache.put(3, 3); // Evicts key 1
  assertEquals(cache.get(1), -1);
});

LRU Cache: put key 1 ... ok (1ms)
LRU Cache: put key 2 ... ok (0ms)
LRU Cache: evict least recently used key ... ok (0ms)
LRU Cache: update value of existing key ... ok (0ms)
LRU Cache: evict after capacity exceeded ... ok (0ms)

ok | 5 passed | 0 failed (2ms)
